In [1]:
from langchain_qwq import ChatQwen
import os

model = ChatQwen(
    model="qwen-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    temperature=0.7,
)

消息（Messages）是 LangChain 中模型上下文的基本单位。它们代表了模型的输入和输出，承载了与大语言模型（LLM）交互时所需的全部内容及元数据，用以表示对话的状态。
消息对象通常包含以下内容：
- 角色：标识消息的类型（例如 system 系统消息、user 用户消息）。
- 内容：表示消息的实际内容（如文本、图片、音频、文档等）。
- 元数据：可选字段，例如响应信息、消息 ID 以及 Token 使用情况。

LangChain 提供了一种标准的消息类型，该类型适用于所有模型提供商，确保了无论调用哪种模型，其行为都保持一致。







# 系统消息

代表一组初始指令，用于设定模型的行为基调，也是通常提到的prompt，可以使用系统消息来确定对话的基调、定义模型的角色，并为模型的回复建立指导方针。

In [2]:
from langchain.messages import SystemMessage

system_msg = SystemMessage("你是一个乐于助人的助手。")



# 用户消息

代表用户的输入和交互。可以包含文本、图片、音频、文件以及任何其他形式的多模态内容。

In [14]:
from langchain.messages import HumanMessage

human_msg = HumanMessage("你好，介绍一下你自己吧")

# AI回复消息

代表一次模型调用的输出结果。它可以包含多模态数据、工具调用，以及之后可供访问的、特定于提供商的元数据。可支持以下参数：
  - `text`：string，消息的文本内容。
  - `content`：string | dict[]，消息的原始内容。
  - `content_blocks`：ContentBlock[]，消息的标准化内容块。
  - `tool_calls`：dict[] | None，模型调用的工具。如果没有调用工具，则为空。
  - `id`：string，消息的唯一标识符（由 LangChain 自动生成或在提供商响应中返回）。
  - `usage_metadata`：dict | None，消息的使用情况元数据，在可用时可包含 token 统计信息。
  - `response_metadata`：ResponseMetadata | None，消息的响应元数据。


content和content_blocks的区别：
- content：原始消息内容，可能是字符串或字典列表。
- content_blocks：将原始内容标准化为结构化的内容块列表，每个块包含类型（如文本、图像等）和内容。

In [16]:
messages = [system_msg, human_msg]
response = model.invoke(messages)  # Returns AIMessage
print(response)

content='你好！很高兴认识你！😊  \n我是通义千问（Qwen），阿里巴巴集团旗下的超大规模语言模型。我能够理解并生成多种语言的文本，比如回答问题、创作文字（如写故事、写公文、写邮件、写剧本）、逻辑推理、编程、多语言支持等。我还具备一定的知识整合与表达能力，可以帮你学习、工作或日常交流中提供帮助。\n\n不过需要说明的是：  \n🔹 我没有真实的情感或意识，但会尽力以友好、耐心、准确的方式回应你；  \n🔹 我的知识截止时间是2024年，因此对之后发生的事件可能不了解；  \n🔹 我不会访问互联网或你的私人数据（除非在特定授权环境中），注重隐私与安全。\n\n如果你有任何问题、想法，或者只是想聊聊——比如想写一首诗、解一道数学题、规划一次旅行、了解某个科学概念……我都很乐意陪你一起探索！✨  \n那么，今天有什么我可以帮你的吗？ 🌟' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 212, 'prompt_tokens': 26, 'total_tokens': 238, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-560587d2-f959-90fa-8ac3-4463ec87ab27', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bffd1-3de5-7410-8e39-46a562467952-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 26, 'output_tokens': 212, 'total_tokens': 238, 'input_token_details': {'cac

In [ ]:
print("text:", response.text)
print("="*60)
print("content:", response.content)
print("="*60)
print("content_blocks:", response.content_blocks)
print("="*60)
print("response_metadata:", response.response_metadata)


如果初始化的时候指定output_version为v1后，content和content_blocks的内容一致，langchain自动在后台完成了这个转化。

In [20]:
model = ChatQwen(
    model="qwen-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    temperature=0.7,
    output_version="v1",
)

In [ ]:
messages = [system_msg, human_msg]
response = model.invoke(messages) 
print("content:", response.content)
print("="*60)
print("content_blocks:", response.content_blocks)

# 工具消息

对于支持调用工具的模型，在发生了工具调用后，AImessages中会包含工具调用（tool calls），工具消息（Tool messages）就是用于将单个工具执行的结果传回给模型的消息类型，工具可以直接返回ToolMessage对象来完成闭环。

工具消息有以下参数：
  - `content`：string（必填），工具调用的字符串化输出结果。
  - `tool_call_id`：string（必填）,此消息所响应的工具调用的 ID。必须与 AIMessage 中工具调用的 ID 匹配。
  - `name`：string,被调用的工具的名称。
  - `artifact`：dict，<span style="color:red">不发送给模型但可通过编程访问的附加数据。</span>

In [ ]:
from langchain.messages import AIMessage
from langchain.messages import ToolMessage

# 在模型发起工具调用之后
# 此处为了简洁，手动创建消息作为演示
ai_message = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather", # 工具名称
        "args": {"location": "宁波"}, # 传递的参数
        "id": "call_123" # 调用ID
    }]
)

# 模拟执行工具并创建结果消息
weather_result = "晴朗，72°F"
tool_message = ToolMessage(
    content=weather_result, # 工具执行返回的内容
    tool_call_id="call_123"  # 必须与上面的调用ID匹配
)

# 继续对话
messages = [
    HumanMessage("宁波的天气怎么样？"),
    ai_message,     # 模型发起的工具调用
    tool_message,   # 工具执行的结果
]
response = model.invoke(messages)  # 模型处理该结果并生成最终回复
print(response.content)

宁波目前天气晴朗，气温约 **22°C**（72°F），体感舒适，适合户外活动。建议注意防晒，可佩戴太阳镜、涂抹防晒霜，并适当补水。

如需了解未来几小时或未来几天的详细预报（如湿度、风速、日出日落时间等），欢迎随时告诉我！ 😊


## artifact

在 LangChain 中，你可以将一些不发送给大语言模型（LLM），但对后续代码处理非常有用的附加数据（Metadata）嵌入到消息对象中。

在工具消息中，通常用于：
- 原始数据传递：将工具调用返回的完整原始 JSON 数据或复杂对象放入 artifact，而只把“人类可读”的摘要文本发送给模型。
- 避免上下文浪费：防止模型去阅读冗长的原始数据，节省 Token

In [ ]:
from langchain_core.messages import ToolMessage
import json

# 假设这是工具调用返回的原始复杂数据
raw_weather_data = {
    "location": "北京",
    "temperature_celsius": 20,
    "condition": "Sunny",
    "humidity": 0.4,
    "wind_speed_kmh": 15,
    "timestamp": "2024-01-27T10:00:00Z"
}

# 创建工具消息
# 1. content: 发送给模型看的（简洁摘要）
# 2. artifact: 不发给模型，但留给代码用的（完整数据）
tool_msg = ToolMessage(
    content="北京今天天气晴朗，气温 20 摄氏度。",
    artifact=raw_weather_data, # 这就是 artifactdict
    name="get_weather" # 工具名称
)

# --- 模拟场景：后续代码如何使用 ---

# 1. 如果你把 tool_msg 发给模型，模型只会看到 content 里的那句话

# 2. 但你的 Python 代码可以直接访问完整的原始数据
#    不需要从文本中去提取温度，直接从 artifact 里拿
full_data = tool_msg.artifact
print(f"原始数据中的风速: {full_data['wind_speed_kmh']} km/h") 
# 输出: 原始数据中的风速: 15 km/h